In [ ]:
import pandas as pd
from config import get_sf_prod
from itables import init_notebook_mode, show

init_notebook_mode(all_interactive=True)

sf = get_sf_prod()

query = """
SELECT Phase__c, Estatus__c, Declined__c, Rejected__c, Formacion__c, Idiomas__c, Experiencia_Conocimientos_Tecnicos__c, Nivel_Seniority__c, Remuneracion__c, No_interes_en_el_sector__c, No_interes_en_la_compania__c, Posicion__c, Ubicacion__c, Salario__c, No_quiere_cambios_otros__c
FROM PDBContactJoin__c 
"""

data = sf.query_all(query)
records = data['records']
df = pd.DataFrame(records).drop(columns='attributes')

In [ ]:
# Handle missing values and ensure correct data types
df['Phase__c'] = df['Phase__c'].fillna('')
df['Estatus__c'] = df['Estatus__c'].fillna('')
df['Declined__c'] = df['Declined__c'].fillna(False).astype(bool)
df['Rejected__c'] = df['Rejected__c'].fillna(False).astype(bool)

# Handle missing values for extra fields
extra_fields = [
    'Formacion__c', 'Idiomas__c', 'Experiencia_Conocimientos_Tecnicos__c', 'Nivel_Seniority__c',
    'Remuneracion__c', 'No_interes_en_el_sector__c', 'No_interes_en_la_compania__c',
    'Posicion__c', 'Ubicacion__c', 'Salario__c', 'No_quiere_cambios_otros__c'
]
for field in extra_fields:
    df[field] = df[field].fillna('')

# Create boolean columns for the states and their combinations
df['discarded'] = df['Estatus__c'] == 'Descartado'
df['declined'] = df['Declined__c']
df['rejected'] = df['Rejected__c']

df['discarded_and_declined'] = df['discarded'] & df['declined']
df['discarded_and_rejected'] = df['discarded'] & df['rejected']
df['declined_and_rejected'] = df['declined'] & df['rejected']

# Create a column to indicate if a record has a discarded reason
df["has_discarded_reason"] = df["Formacion__c"] | df["Idiomas__c"] | df["Experiencia_Conocimientos_Tecnicos__c"] | df["Nivel_Seniority__c"]
df["has_rejection_reason"] = df["Remuneracion__c"] | df["No_interes_en_el_sector__c"] | df["No_interes_en_la_compania__c"] | df["Posicion__c"] | df["Ubicacion__c"] | df["Salario__c"] | df["No_quiere_cambios_otros__c"]



In [ ]:
result_rows = []
phases = sorted(df['Phase__c'].unique())
for phase in phases:
    df_phase = df[df['Phase__c'] == phase]
    total = len(df_phase)
    if total == 0:
        continue  # Skip phases with no data

    counts = {
        'Discarded (New) %': df_phase['discarded'].sum(),
        'Rejected (Old) %': df_phase['rejected'].sum(),
        'Declined (Old) %': df_phase['declined'].sum(),
        'Discarded and Declined %': df_phase['discarded_and_declined'].sum(),
        'Discarded and Rejected %': df_phase['discarded_and_rejected'].sum(),
        'Declined and Rejected %': df_phase['declined_and_rejected'].sum(),
        'Discarded with reason %': df_phase['has_discarded_reason'].sum(),
        'Rejected with reason %': df_phase['has_rejection_reason'].sum(),
    }

    percentages = {k: (v / total) * 100 for k, v in counts.items()}
    result_row = {'Phase': phase, 'Total': total}
    result_row.update(percentages)
    result_rows.append(result_row)

# Create a DataFrame for the results
result_df = pd.DataFrame(result_rows)

# Round percentages to two decimal places and format
percentage_columns = [col for col in result_df.columns if '%' in col]
result_df[percentage_columns] = result_df[percentage_columns].round(2)

# Optionally, format percentages with '%' sign
for col in percentage_columns:
    result_df[col] = result_df[col].map('{:.2f}%'.format)

# Set 'Phase' as the index
result_df = result_df.set_index('Phase')

# Display the result
show(result_df)